In [7]:
import openai
import os
import sys
sys.path.append("../")
from magpie.prompt_planner.prompts import mp_prompt_thinker_coder_muk as mptc
from magpie.prompt_planner import conversation

In [2]:
import numpy as np
from PIL import Image
import base64
import io
import cv2 

def encode_image(img):
    # Convert NumPy array to base64 encoded string
    return base64.b64encode(img).decode('utf-8')


i = encode_image(np.array(Image.open("static/favicon.jpg")))

In [10]:
MODEL = "gpt-3.5-turbo"
import dataclasses
from typing import Any

In [12]:
@dataclasses.dataclass(frozen=True)
class TaskConfig:
  client: None
  prompts: dict[str, type[Any]]

ALL_TASKS = {
    'magpie': TaskConfig(
        client=None,
        prompts={
            'thinker_coder': mptc.PromptThinkerCoder,
        },
    ),
}


AttributeError: module 'magpie.prompt_planner.prompts.mp_prompt_thinker_coder_muk' has no attribute 'PromptThinkerCoder'